# Heterogeneous GNN

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
from torch_geometric.data import HeteroData, DataLoader, Dataset
from torch_geometric.nn import RGCNConv
import torch.nn.functional as F
from torch_geometric.nn import SAGEConv, to_hetero, GCNConv
from torch_geometric.nn import HeteroConv, GCNConv, SAGEConv, GATConv, GraphConv, Linear
from torch_geometric.nn import global_mean_pool
from torch_geometric.utils import trim_to_layer
from torch_geometric.loader import DataLoader
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

class InfernoDataset(Dataset):
    def __init__(self, data_list):
        super(InfernoDataset, self).__init__()
        self.data_list = data_list

    def len(self):
        return len(self.data_list)

    def get(self, idx):
        return self.data_list[idx]

In [3]:
dataset = torch.load('data/inferno_graph_dataset.pt')
dataset.len()

64686

In [4]:
dataset2 = torch.load('data/inferno_graph_dataset_2.pt')
dataset2.len()

165182

In [5]:
dataset3 = torch.load('data/inferno_graph_dataset_3.pt')
dataset3.len()

61479

In [6]:
64686+165182+61479

291347

In [2]:
dataset = torch.load('data/blast-premier-fall-groups-2023-evil-geniuses-vs-g2-bo3-m1-inferno.pt')
data = dataset[0]
data

HeteroData(
  y={
    roundNum=1.0,
    sec=0.0,
    team1AliveNum=5.0,
    team2AliveNum=5.0,
    CTwinsRound=1
  },
  player={ x=[10, 44] },
  map={ x=[181, 3] },
  (map, connected_to, map)={ edge_index=[2, 204] },
  (player, closest_to, map)={ edge_index=[2, 10] }
)

In [3]:
class HeterogeneousGNN(torch.nn.Module):

    def __init__(self, hidden_channels, num_layers, edge_types):
        super().__init__()
        
        torch.manual_seed(42)
        
        self.convs = torch.nn.ModuleList()
        for layernum in range(num_layers):
            conv = HeteroConv({
                    edge_type: SAGEConv((-1, -1), hidden_channels)
                    for edge_type in edge_types
                }, aggr='sum')
            self.convs.append(conv)
        self.lin1 = Linear(-1, 128)
        self.lin2 = Linear(128, 1)

    def forward(self, x_dict, edge_index_dict):
        for conv in self.convs:
            temp = conv(x_dict, edge_index_dict)
            x_dict['map'] = temp['map']
            x_dict = {key: x.relu() for key, x in x_dict.items()}
            
        x = torch.cat([torch.flatten(x_dict['player']), torch.flatten(x_dict['map'])])
        x = self.lin1(x).relu()
        x = self.lin2(x).sigmoid()
        return x

model = HeterogeneousGNN(hidden_channels=20, num_layers=4, edge_types=data.edge_types)
print(model);

HeterogeneousGNN(
  (convs): ModuleList(
    (0-3): 4 x HeteroConv(num_relations=2)
  )
  (lin1): Linear(-1, 128, bias=True)
  (lin2): Linear(128, 1, bias=True)
)


C:\Users\szmid\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\torch_geometric\nn\conv\hetero_conv.py:62: UserWarning: There exist node types ({'player'}) whose representations do not get updated during message passing as they do not occur as destination type in any edge type. This may lead to unexpected behavior.
  warnings.warn(


In [7]:
train_loader = DataLoader(dataset[:960], batch_size=1, shuffle=True)
val_loader = DataLoader(dataset[960:], batch_size=1, shuffle=True)
data = dataset[0]

with torch.no_grad():  # Initialize lazy modules.
     out = model(data.x_dict, data.edge_index_dict)
     print(out)

tensor([1.1921e-08])


In [10]:
model = HeterogeneousGNN(hidden_channels=20, num_layers=10, edge_types=data.edge_types).to('cuda')
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
loss_function = torch.nn.BCELoss()

def train():
    model.train()
    optimizer.zero_grad()

    for data in train_loader:  # Iterate in batches over the training dataset.
        data.to('cuda')
        out = model(data.x_dict, data.edge_index_dict).to(torch.float32)  # Perform a single forward pass.
        target = data.y['CTwinsRound'].to(torch.float32)
        loss = loss_function(out, target)  # Compute the loss.
        loss.backward()  # Derive gradients.
        optimizer.step()  # Update parameters based on gradients.
        

def test(loader):
     model.eval()

     correct = 0
     for data in loader:  # Iterate in batches over the training/test dataset.
         out = model(data.x, data.edge_index, data.batch)  
         pred = out.argmax(dim=1)  # Use the class with highest probability.
         correct += int((pred == data.y).sum())  # Check against ground-truth labels.
     return correct / len(loader.dataset)  # Derive ratio of correct predictions.


def validate(val_loader):
    model.eval()  # Átkapcsoljuk a modellt értékelési üzemmódba.
    total_loss = 0
    correct_predictions = 0
    total_samples = 0

    with torch.no_grad():
        for data in val_loader:  # Iterálunk a validációs adatokon.
            data.to('cuda')
            out = model(data.x_dict, data.edge_index_dict).to(torch.float32)
            target = data.y['CTwinsRound'].to(torch.float32)
            loss = loss_function(out, target)
            total_loss += loss.item()
            total_samples += len(target)

            # Ellenőrizzük a helyes előrejelzéseket (például egy bináris probléma esetében).
            predictions = (out > 0.5).float()
            correct_predictions += (predictions == target).sum().item()

    # Kiszámítjuk az átlagos veszteséget és a pontosságot.
    avg_loss = total_loss / len(val_loader)
    accuracy = correct_predictions / total_samples

    return avg_loss, accuracy



for epoch in range(1, 5):
    train()
    train_acc = validate(val_loader)
    print(epoch, '- ', train_acc)

1 -  (66.5625, 0.334375)
2 -  (66.5625, 0.334375)


KeyboardInterrupt: 